<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/time_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN(Recurrent Neural Network)


In [ ]:
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next): # dh_next는 흘러들어온 미분
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2) # 활성화 함수 tanh에 대한 미분.(1-y^2)
        db = np.sum(dt, axis=0) # repeat node에 대한 역전파.
        dWh = np.dot(h_prev.T, dt) # dot 연산에 대한 역전파
        dh_prev = np.dot(dt, Wh.T) # dot 연산에 대한 역전파
        dWx = np.dot(x.T, dt) # dot 연산에 대한 역전파
        dx = np.dot(dt, Wx.T) # dot에 연산에 대한 역전파

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

# TimeRNN(미니배치, TruncatedBPTT)

In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False): # stateful은 TruncatedBPTT의 시간블록의 출력인 히든스테이트를 다음 시간블록으로 전달할지 말지 결정한다.
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape # 배치 크기, 시간블록 크기, embedding층을 거친 벡터 차원.
        D, H = Wx.shape # data 는 D, D를 은닉층 H 으로 보내는 Weight matirx.

        self.layers = [] # RNN 시간블록을 저장할 리스트.
        hs = np.empty((N, T, H), dtype='f') # 3차원 텐서 생성. empty는 랜덤한 값을 생성함. 히든스테이트들을 저장할 것.

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f') # 히든스테이를 다음 블록으로 전달하지 않을 경우 제로행렬을 전달.

        for t in range(T): # T는 타임사이즈.
            layer = RNN(*self.params) # 브라켓을 떼고 RNN인슨턴스 생성에 필요한 파라미터를 입력.
            self.h = layer.forward(xs[:, t, :], self.h) # 시간블록 선택(data), 히든스테이트 선택.
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs # 히든 스테이트들의 모임을 반환.

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape # 흘러들어온 미분의 shape.
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0 # dh는 다음 시간블록에서 흘러들어온 미분.
        grads = [0, 0, 0] # gradient를 저장할 리스트.
        for t in reversed(range(T)): # 역전파이므로 마지막 시간블록부터 시작.
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh) # 흘러들어온 두 미분을 합한다.
            dxs[:, t, :] = dx

            for i, grad in enumerate(layer.grads): # gardient 저장.
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None

# Time Embedding

In [ ]:
class TimeEmbedding:
    def __init__(self, W): # bias는 없고 W는 각 단어들의 벡터표현이 각 행에 저장돼어 있음.
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W

    def forward(self, xs): # 시간 블럭으로 묶인 단어 묶음.
        N, T = xs.shape
        V, D = self.W.shape # vocab(단어 개수)사이즈 * vector의 크기

        out = np.empty((N, T, D), dtype='f') # xs와 동일한 사이즈의 행렬에 원소가 벡터로 변함.(각 단어의 ID를 벡터로 변환)
        self.layers = [] # embedding 인스턴스들을 저장

        for t in range(T): # T는 시간블럭의 길이
            layer = Embedding(self.W)
            out[:, t, :] = layer.forward(xs[:, t])
            self.layers.append(layer)

        return out

    def backward(self, dout):
        N, T, D = dout.shape # dout의 shape은 out의 shape와 동일하다.

        grad = 0
        for t in range(T):
            layer = self.layers[t]
            layer.backward(dout[:, t, :])
            grad += layer.grads[0] # 모든 gredient를 더한후 업데이트한다.

            # ? 업데이트 과정은?

        self.grads[0][...] = grad
        return None

# Time Affine

In [ ]:
class TimeAffine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x): # TimeRNN 층에서 올라온 hidden states 묶음.
        N, T, D = x.shape # 베치사이즈 * 타임사이즈 * 히든스테이트 벡터의 차원.
        W, b = self.params

        rx = x.reshape(N*T, -1) # 히든 스테이트 하나 하나 affine 변환을 하는 것 보다 행렬로 바꿔서 한번에 연산하는 것 이 빠름.
        out = np.dot(rx, W) + b # W 의 사이즈는 히든사이즈 * voc사이즈.( 각 단어의 스코어를 부여해야 하므로)
        self.x = x
        return out.reshape(N, T, -1) # 원래형태로 복원

    def backward(self, dout): # dout은 Time Softmax층에서 흘러들어온 미분.
        x = self.x
        N, T, D = x.shape
        W, b = self.params

        dout = dout.reshape(N*T, -1) # 순전파에서와의 같은 이유로 3차원텐서를 2차원 행렬로 변환.
        rx = x.reshape(N*T, -1) # rx는 TimeRNN에서 올라온 히든 스테이트 텐서를 행렬로 변환.

        db = np.sum(dout, axis=0) # repeat노드의 역전파
        dW = np.dot(rx.T, dout)
        dx = np.dot(dout, W.T)
        dx = dx.reshape(*x.shape)

        self.grads[0][...] = dW # 업데이트
        self.grads[1][...] = db

        return dx # 밑으로 흘려보냄

# TimeSoftmaxWithLoss

In [ ]:
class TimeSoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], [] # S.L층은 학습해야할 파라미터가 없음.
        self.cache = None
        self.ignore_label = -1 # 특정 라벨을 무시.

    def forward(self, xs, ts): # ts는 label
        N, T, V = xs.shape # 단어에 대한 스코어.

        if ts.ndim == 3:  # 정답 레이블이 원핫 벡터인 경우
            ts = ts.argmax(axis=2) # 원핫인코딩 되기전으로 수정.

        mask = (ts != self.ignore_label)

        # 배치용과 시계열용을 정리(reshape)
        xs = xs.reshape(N * T, V) # 3차언텐서를 행렬로.
        ts = ts.reshape(N * T) # flatten
        mask = mask.reshape(N * T)

        ys = softmax(xs)
        ls = np.log(ys[np.arange(N * T), ts])# 슬라이싱 0,3  1,2  ...
        ls *= mask  # ignore_label에 해당하는 데이터는 손실을 0으로 설정
        loss = -np.sum(ls)
        loss /= mask.sum()  # corss entropy 구하는 과정

        self.cache = (ts, ys, mask, (N, T, V))
        return loss

    def backward(self, dout=1): # 마지막층이므로 흘러들어온 미분은 1
        ts, ys, mask, (N, T, V) = self.cache

        dx = ys
        dx[np.arange(N * T), ts] -= 1 # 슬라이싱, 빼기. S.L의 역전파
        dx *= dout
        dx /= mask.sum() # 개수로 나눠줌
        dx *= mask[:, np.newaxis]  # 벡터를 열벡터로하는 행렬로 변환. ignore_label에 해당하는 데이터는 기울기를 0으로 설정

        dx = dx.reshape((N, T, V))

        return dx